In [1]:
from pathlib import Path
import os
from datetime import datetime
from datetime import timedelta
from send2trash import send2trash
import shutil

In [3]:
folder_path = '/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/'
folder_dir = Path(folder_path)

In [21]:
# step 1: access files
RECENT_DAYS  = 30
most_recent = datetime.now() - timedelta(days = RECENT_DAYS)
num_file = 0
num_subdir = 0
dir_size = 0
num_recent_file = 0
file_list = []
subdir_name_list = []

for entry in folder_dir.iterdir():
    if entry.is_file():
        file_list.append(entry)
        num_file += 1
        mtime = datetime.fromtimestamp(entry.stat().st_mtime)
        if mtime > most_recent:
            num_recent_file += 1

    if entry.is_dir():
        num_subdir += 1
        subdir_name_list.append(entry.name)
    
    
for f in folder_dir.glob('**/*'):
    if f.is_file():
        dir_size += f.stat().st_size

print('Num files: %d' % num_file)
print('Recent files: %d' % num_recent_file)
print('Num subdirs: %d' % num_subdir)
print('Dir size: %d' % dir_size)

Num files: 20
Recent files: 12
Num subdirs: 3
Dir size: 19192530


In [5]:
def create_dir(current_dir, name):
    path = current_dir.absolute().as_posix() + '/' + name
    p = Path(path)
    try:
        p.mkdir()
    except FileExistsError as exc:
        print('Folder %s already exists!'%name)
    return(path)

In [33]:
def print_window(file_name):
    print('---'*20)
    print('0: delete, 1: Pass, 2: Move to... 3: Undo, 4: Done, 5: Preview file(only for Mac)')
    print('   ')
    print('>> File: %s' % file_name)


In [32]:
def get_action(file_path):
    input_flag = True
    while(input_flag):
        action = input() #input('0: delete, 1: Pass, 2: Move to... 3: Undo, 4: Done')
        if action in ['0','1','2','3','4']:
            input_flag = False
        elif action == '5':
            cmd = "qlmanage -p " + file_path
            os.popen(cmd).read()

        else:
            print('Invalid input "%s", do it again' %action)
    return(action)

In [26]:
def move_file(file_path, target_folder_path,target_path):
    
    # move file to target file folder
    new_path = shutil.move(file_path,target_folder_path)

    # add the path of the file after action
    target_path.append(new_path)
    

def create_move_dir(current_dir):
    dir_name = input('Input folder name:')
    temp_path = create_dir(current_dir, dir_name)
    return(dir_name,temp_path)

def move_action(file_path, current_dir, move_dirs_path, move_dirs_name, target_path):

    if not move_dirs_path:
        print('No move folders yet. Create a new folder: ')
        dir_name,temp_path = create_move_dir(current_dir)
        move_dirs_name.append(dir_name)
        move_dirs_path.append(temp_path)
        move_file(file_path, temp_path,target_path)
    
    else:
        print('Move folders: ')
        for ii, l in enumerate(move_dirs_name):
            print('%d  %s' %(ii, l))
        print('-1: create a new folder')

        temp_flag = True
        while(temp_flag):
            try: 
                move_dir_id = int(input('Choose which folder to move:'))
            except:
                print('Invalid input')
            else:
                temp_flag = False

        if move_dir_id >= 0:
            temp_path = move_dirs_path[move_dir_id]
            dir_name = move_dirs_name[move_dir_id]
            move_file(file_path, temp_path, target_path)
            action_list.append('2')
        else:
            dir_name,temp_path = create_move_dir(current_dir)
            move_dirs_name.append(dir_name)
            move_dirs_path.append(temp_path)
            move_file(file_path, temp_path,target_path)
            
    return(dir_name)
            

def undo_action(previous_action, current_path, old_dir):

    if previous_action in ['0','2']:
        new_path = shutil.move(current_path,old_dir)
        temp_file = Path(new_path)

    elif previous_action == '1':
        temp_file = Path(current_path)

    return(temp_file)
            
def done_action(trash_path):
    # move trash files to reycle bin and delete trash
    trash_dir = Path(trash_path)
    for f in trash_dir.iterdir():
        send2trash(f.absolute().as_posix())
    trash_dir.rmdir()


In [34]:
# step 2: manipulate files
# delete, redo, skip, move
# build a trash dir to store all the trash in the end more files to recycle bin and delete trash dir
move_dirs_path = []
move_dirs_name = []
action_list = []
target_path = []
trash_path = create_dir(folder_dir, 'trash_bin')

#for entry in current_dir.iterdir():
while file_list:

    entry = file_list.pop()

    if entry.is_file():
        file_name = entry.name
        # get file path
        file_path = entry.absolute().as_posix()

        print_window(file_name)
        action = get_action(file_path)
        
        if action == '0':
            # delete
            move_file(file_path, trash_path, target_path)
            action_list.append(action)

            print('>> %s deleted!'%file_name)

        elif action == '1':
            # pass
            target_path.append(file_path)
            action_list.append(action)
            print('>> %s passed!'%file_name)

        elif action == '2':
            # Move 
            move_folder_name = move_action(file_path, folder_dir, move_dirs_path, move_dirs_name,target_path)
            action_list.append(action)
            print('>> %s moved to folder[%s]!'%(file_name, move_folder_name))
              
        elif action == '3':
            # undo the last action
            # return to previous file
            file_list.append(entry)
            if action_list:
                previous_action = action_list.pop()
                temp_path = target_path.pop()
                temp_entry = undo_action(previous_action, temp_path, folder_dir)
                file_list.append(temp_entry)
                print('>> Undo the last action.')
            else:
                print('No action to undo!')
            

        else:
            # done
            done_action(trash_path)
            print('>> Done and quit')
            break

        
        

        
        



------------------------------------------------------------
0: delete, 1: Pass, 2: Move to... 3: Undo, 4: Done, 5: Preview file(only for Mac)
   
>> File: uh_2d.png
>> Done and quit


In [102]:
a = Path('fsdfsf/sfsd')

In [103]:
a.is_file()

False

In [31]:
# QuickLook
name = 'err_uh_3d.png'
cmd = "qlmanage -p " + name
os.popen(cmd).read()

'Testing Quick Look preview with files:\n\terr_uh_3d.png\n'

In [93]:
import sys

In [94]:
sys.platform

'darwin'

In [ ]:
# 1. create a square window with options
# 2. write each action as function to clean the code
# 3. give a status bar